In [ ]:
%install '.package(path: "$cwd/FastaiNotebook_00_load_data")' FastaiNotebook_00_load_data

# Conform Array to Differentiable

In [ ]:
//export

extension Array where Element: Differentiable {
    /// Views the array as the differentiable product manifold of `Element` with itself `count` times.
    public struct DifferentiableView: Differentiable {
        /// The array that we are viewing.
        public var base: [Element]

        /// Construct a view of the given array.
        public init(_ base: [Element]) { self.base = base }
        
        // MARK: - Differentiable conformance.

        public typealias TangentVector = Array<Element.TangentVector>.DifferentiableView
        public typealias CotangentVector = Array<Element.CotangentVector>.DifferentiableView
        public typealias AllDifferentiableVariables = Array<Element.AllDifferentiableVariables>.DifferentiableView
        
        public var allDifferentiableVariables: AllDifferentiableVariables {
            get {
                return AllDifferentiableVariables(base.map { $0.allDifferentiableVariables })
            }
            set(v) {
                precondition(base.count == v.base.count, "count mismatch")
                for i in base.indices {
                    base[i].allDifferentiableVariables = v.base[i]
                }
            }
        }

        public func moved(along direction: TangentVector) -> DifferentiableView {
            precondition(base.count == direction.base.count, "count mismatch")
            return DifferentiableView(zip(base, direction.base).map { $0.moved(along: $1) })
        }

        public func tangentVector(from cotangentVector: CotangentVector) -> TangentVector {
            precondition(base.count == cotangentVector.base.count, "count mismatch")
            return TangentVector(zip(base, cotangentVector.base).map {
                (selfElement, cotangentVectorElement) in
                selfElement.tangentVector(from: cotangentVectorElement)
            })
        }
    }
}

extension Array.DifferentiableView: KeyPathIterable {}

extension Array.DifferentiableView: Equatable where Element: Equatable {
    public static func == (lhs: Array.DifferentiableView, rhs: Array.DifferentiableView) -> Bool {
        return lhs.base == rhs.base
    }
}

/// Makes `Array.DifferentiableView` additive as the product space.
///
/// Note that `Array.DifferentiableView([])` is the zero in the product spaces of all counts.
extension Array.DifferentiableView: AdditiveArithmetic where Element: AdditiveArithmetic {
    public static var zero: Array.DifferentiableView { return Array.DifferentiableView([]) }
    public static func + (lhs: Array.DifferentiableView, rhs: Array.DifferentiableView) -> Array.DifferentiableView {
        precondition(lhs.base.count == 0 || rhs.base.count == 0 || lhs.base.count == rhs.base.count, "count mismatch")
        if lhs.base.count == 0 {
            return rhs
        }
        if rhs.base.count == 0 {
            return lhs
        }
        return Array.DifferentiableView(zip(lhs.base, rhs.base).map(+))
    }
    public static func - (lhs: Array.DifferentiableView, rhs: Array.DifferentiableView) -> Array.DifferentiableView {
        precondition(lhs.base.count == 0 || rhs.base.count == 0 || lhs.base.count == rhs.base.count, "count mismatch")
        if lhs.base.count == 0 {
            return rhs
        }
        if rhs.base.count == 0 {
            return lhs
        }
        return Array.DifferentiableView(zip(lhs.base, rhs.base).map(-))
    }
}

/// Makes `Array` differentiable as the product manifold of `Element` with itself `count` times.
///
/// This is a convenience wrapper around `Array.DifferentiableView`.
extension Array: Differentiable where Element: Differentiable {
    public typealias TangentVector = Array<Element.TangentVector>.DifferentiableView
    public typealias CotangentVector = Array<Element.CotangentVector>.DifferentiableView
    public typealias AllDifferentiableVariables = Array<Element.AllDifferentiableVariables>.DifferentiableView

    public var allDifferentiableVariables: AllDifferentiableVariables {
        get {
            return DifferentiableView(self).allDifferentiableVariables
        }
        set(v) {
            var view = DifferentiableView(self)
            view.allDifferentiableVariables = v
            self = view.base
        }
    }

    public func moved(along direction: TangentVector) -> Array {
        return DifferentiableView(self).moved(along: direction).base
    }

    public func tangentVector(from cotangentVector: CotangentVector) -> TangentVector {
        return DifferentiableView(self).tangentVector(from: cotangentVector)
    }
}

# Helpers making it easier to use arrays in models

In [ ]:
//export
import TensorFlow

In [ ]:
//export
extension Array: Layer where Element: Layer, Element.Input == Element.Output {
    public typealias Input = Element.Input
    public typealias Output = Element.Output
    
    @differentiable(vjp: _vjpApplied)
    public func applied(to input: Input, in context: Context) -> Output {
        var activation = input
        for layer in self {
            activation = layer.applied(to: activation, in: context)
        }
        return activation
    }
    
    public func _vjpApplied(_ input: Input, in context: Context)
        -> (Output, (Output.CotangentVector) -> (Array.CotangentVector, Input.CotangentVector))
    {
        var activation = input
        var pullbacks: [(Input.CotangentVector) -> (Element.CotangentVector, Input.CotangentVector)] = []
        for layer in self {
            let (newActivation, newPullback) = layer.valueWithPullback(at: activation) {
                $0.applied(to: $1, in: context)
            }
            activation = newActivation
            pullbacks.append(newPullback)
        }
        func pullback(_ v: Input.CotangentVector) -> (Array.CotangentVector, Input.CotangentVector) {
            var activationGradient = v
            var layerGradients: [Element.CotangentVector] = []
            for pullback in pullbacks.reversed() {
                let (newLayerGradient, newActivationGradient) = pullback(activationGradient)
                activationGradient = newActivationGradient
                layerGradients.append(newLayerGradient)
            }
            return (Array.CotangentVector(layerGradients.reversed()), activationGradient)
        }
        return (activation, pullback)
    }
}

In [ ]:
//export

// The optimizers in DeepLearning are incompatible with Array fields, so we define our own here.

public class SimpleSGD<Model: Layer, Scalar: TensorFlowFloatingPoint>: Optimizer
    where Model.AllDifferentiableVariables == Model.CotangentVector {
    /// The learning rate.
    public var learningRate: Scalar

    public init(
        learningRate: Scalar = 0.01
    ) {
        precondition(learningRate >= 0, "Learning rate must be non-negative")
        self.learningRate = learningRate
    }

    public func update(_ model: inout Model.AllDifferentiableVariables,
                       along direction: Model.CotangentVector) {
        for kp in model.recursivelyAllWritableKeyPaths(to: Tensor<Scalar>.self) {
            model[keyPath: kp] -= learningRate * direction[keyPath: kp]
        }
    }
}

# Export

In [ ]:
import FastaiNotebook_00_load_data
import Path
notebookToScript(fname: (Path.cwd / "01c_array_differentiable.ipynb").string)